In [1]:
import pandas as pd
from pathlib import Path
import numpy as np

In [2]:
DATA_DIR = Path("data/processing_checkpoint")
path_clean_dataset = DATA_DIR / "01_cleaned_train.parquet"

In [3]:
df_clean_churn = pd.read_parquet(path_clean_dataset)

In [4]:
df_clean_churn.sample(5)

,gender,level,userId,page,sessionId,itemInSession,length,song,artist,time,registration,metropolitan_area,region,operating_system,browser
2904987,F,paid,1882293,NextSong,105728,23,266.68363,Always All Ways [apologies_ Glances And Messed...,Lostprophets,2018-10-21 16:45:20,2018-09-28 06:40:30,Greensboro-High Point,NC,Windows,Chrome
14040019,M,paid,1863821,NextSong,16743,85,168.90730,I Second That Emotion,Smokey Robinson & The Miracles,2018-10-22 20:00:29,2018-07-03 11:15:56,Houston-The Woodlands-Sugar Land,TX,iPad,Safari
479193,F,free,1090733,NextSong,71022,65,196.28363,DONTTRUSTME [BENNYBLANCOREMIX] FEATURINGKIDCUD...,3OH!3,2018-10-15 08:23:51,2018-09-26 09:05:41,Las Vegas-Henderson-Paradise,NV,Windows,Edge
1122775,F,paid,1916318,NextSong,81670,76,191.16363,Pdsa,Samy Deluxe,2018-10-15 20:13:04,2018-09-23 22:05:34,Kokomo,IN,iPhone,Safari
4252520,F,free,1801590,NextSong,10144,13,540.94322,Get Your Grind On (feat. Christopher Just) (Xm...,Larry Tee,2018-10-01 14:19:04,2018-09-15 11:31:57,Los Angeles-Long Beach-Anaheim,CA,Macintosh,Safari


In [5]:
df_users_base = df_clean_churn[["userId", "gender", "registration", "operating_system", "browser"]].drop_duplicates(subset=["userId"])

In [6]:
len(df_users_base)

19140

In [7]:
df_users_base = df_users_base.set_index("userId")

In [8]:
df_unique_artists = (
    df_clean_churn.groupby("userId")["artist"].nunique().reset_index().rename(columns={"artist": "num_unique_artists"})
)
df_unique_artists = df_unique_artists.set_index("userId")
df_users_base = df_users_base.join(df_unique_artists)

In [9]:
df_users_base.sample(5)

,gender,registration,operating_system,browser,num_unique_artists
userId,,,,,
1257216,M,2018-08-11 12:26:15,Windows,Chrome,420
1869650,M,2018-08-09 01:13:03,Windows,Chrome,859
1470904,M,2018-09-28 10:58:04,Windows,Chrome,975
1784723,F,2018-05-30 13:44:22,Windows,Chrome,412
1940569,M,2018-09-10 11:40:29,Windows,Chrome,793


In [10]:
df_page_counts = (
    df_clean_churn.groupby("userId")["page"]
    .value_counts()
    .unstack(fill_value=0)
)

df_page_counts.columns = [f"count_{col.replace(' ', '_').lower()}" for col in df_page_counts.columns]

In [11]:
df_users_base = df_users_base.join(df_page_counts)
df_users_base.sample(5)

,gender,registration,operating_system,browser,num_unique_artists,count_about,count_add_friend,count_add_to_playlist,count_cancel,count_cancellation_confirmation,...,count_logout,count_nextsong,count_roll_advert,count_save_settings,count_settings,count_submit_downgrade,count_submit_upgrade,count_thumbs_down,count_thumbs_up,count_upgrade
userId,,,,,,,,,,,,,,,,,,,,,
1745993,F,2018-09-08 14:27:58,Windows,Chrome,6,0,0,0,0,0,...,1,6,4,1,1,0,0,0,0,0
1397827,F,2018-06-23 21:22:04,Windows,Edge,108,0,3,5,0,0,...,6,111,5,2,3,0,0,2,5,0
1256032,F,2018-09-14 20:42:29,Macintosh,Chrome,28,0,0,1,1,1,...,0,27,2,0,0,0,0,1,1,0
1853100,F,2018-09-08 01:40:01,Windows,Firefox,515,0,4,21,0,0,...,4,612,8,1,2,0,1,6,34,1
1993807,F,2018-09-09 01:38:16,Macintosh,Safari,940,3,20,39,0,0,...,21,1260,25,2,8,1,1,38,69,2


In [12]:
df_session_count = (
    df_clean_churn.groupby("userId")["sessionId"]
    .nunique()
    .reset_index(name="count_total_sessions")
    .set_index("userId")
)

df_users_base = df_users_base.join(df_session_count)


In [13]:
df_users_base.sample(5)

,gender,registration,operating_system,browser,num_unique_artists,count_about,count_add_friend,count_add_to_playlist,count_cancel,count_cancellation_confirmation,...,count_nextsong,count_roll_advert,count_save_settings,count_settings,count_submit_downgrade,count_submit_upgrade,count_thumbs_down,count_thumbs_up,count_upgrade,count_total_sessions
userId,,,,,,,,,,,,,,,,,,,,,
1422286,M,2018-08-06 19:10:35,iPhone,Safari,477,1,7,8,0,0,...,556,58,1,3,0,0,5,35,5,15
1344712,M,2018-08-10 00:11:03,Windows,Firefox,63,0,2,0,1,1,...,65,7,0,0,0,0,1,2,1,5
1976447,M,2018-08-05 22:42:50,Macintosh,Safari,73,0,3,3,0,0,...,77,5,0,2,0,0,1,5,1,4
1435095,F,2018-09-10 05:54:57,Windows,Firefox,75,0,0,3,0,0,...,75,6,0,0,0,0,1,1,0,4
1086328,M,2018-09-23 05:14:00,Macintosh,Chrome,20,1,1,1,1,1,...,19,1,0,0,0,0,2,0,0,1


In [14]:
# Build user lifecycle

df_last_time = (
    df_clean_churn.groupby("userId")["time"]
    .max()
    .reset_index(name="last_time")
    .set_index("userId")
)

df_users_base = df_users_base.join(df_last_time)
df_users_base["user_lifecycle_h"] = df_users_base["last_time"] - df_users_base["registration"]
df_users_base["user_lifecycle_h"] = df_users_base["user_lifecycle_h"].dt.total_seconds() / 3600

df_users_base.head()

,gender,registration,operating_system,browser,num_unique_artists,count_about,count_add_friend,count_add_to_playlist,count_cancel,count_cancellation_confirmation,...,count_save_settings,count_settings,count_submit_downgrade,count_submit_upgrade,count_thumbs_down,count_thumbs_up,count_upgrade,count_total_sessions,last_time,user_lifecycle_h
userId,,,,,,,,,,,,,,,,,,,,,
1749042,M,2018-08-08 13:22:21,Windows,Chrome,797,5,18,33,1,1,...,2,7,0,0,9,51,0,9,2018-10-21 01:16:24,1763.900833
1563081,F,2018-09-21 03:25:18,Macintosh,Chrome,437,1,5,14,0,0,...,0,2,0,0,5,25,0,6,2018-11-03 02:00:13,1030.581944
1697168,F,2018-09-08 13:48:25,Macintosh,Firefox,1117,1,22,44,0,0,...,2,12,1,1,16,83,7,34,2018-11-19 23:59:55,1738.191667
1222580,M,2018-08-16 02:31:00,Macintosh,Safari,1252,8,35,50,1,1,...,1,11,2,2,17,85,5,20,2018-10-30 23:17:30,1820.775000
1714398,F,2018-09-19 18:23:35,Windows,Chrome,1102,0,22,51,0,0,...,2,9,0,0,19,79,0,14,2018-11-19 05:16:55,1450.888889


In [15]:
df_length = (
    df_clean_churn.groupby("userId")["length"].sum()
)
df_length = (
    df_length
    .reset_index(name="ttl_length")
    .set_index("userId")
)
df_users_base = df_users_base.join(df_length)
df_users_base.head(5)


,gender,registration,operating_system,browser,num_unique_artists,count_about,count_add_friend,count_add_to_playlist,count_cancel,count_cancellation_confirmation,...,count_settings,count_submit_downgrade,count_submit_upgrade,count_thumbs_down,count_thumbs_up,count_upgrade,count_total_sessions,last_time,user_lifecycle_h,ttl_length
userId,,,,,,,,,,,,,,,,,,,,,
1749042,M,2018-08-08 13:22:21,Windows,Chrome,797,5,18,33,1,1,...,7,0,0,9,51,0,9,2018-10-21 01:16:24,1763.900833,256456.28661
1563081,F,2018-09-21 03:25:18,Macintosh,Chrome,437,1,5,14,0,0,...,2,0,0,5,25,0,6,2018-11-03 02:00:13,1030.581944,123645.65861
1697168,F,2018-09-08 13:48:25,Macintosh,Firefox,1117,1,22,44,0,0,...,12,1,1,16,83,7,34,2018-11-19 23:59:55,1738.191667,386582.82117
1222580,M,2018-08-16 02:31:00,Macintosh,Safari,1252,8,35,50,1,1,...,11,2,2,17,85,5,20,2018-10-30 23:17:30,1820.775000,452234.08024
1714398,F,2018-09-19 18:23:35,Windows,Chrome,1102,0,22,51,0,0,...,9,0,0,19,79,0,14,2018-11-19 05:16:55,1450.888889,384934.72168


In [16]:
df_item_per_session = (
    df_clean_churn.groupby("userId")["itemInSession"].max() / df_users_base["count_total_sessions"]
)

df_item_per_session = (
    df_item_per_session
    .reset_index(name="item_per_session")
    .set_index("userId")
)

df_users_base = df_users_base.join(df_item_per_session)


In [17]:
df_users_base["frequency"] = df_users_base["count_total_sessions"] / df_users_base["user_lifecycle_h"]

In [18]:
df_users_base.sample(5)

,gender,registration,operating_system,browser,num_unique_artists,count_about,count_add_friend,count_add_to_playlist,count_cancel,count_cancellation_confirmation,...,count_submit_upgrade,count_thumbs_down,count_thumbs_up,count_upgrade,count_total_sessions,last_time,user_lifecycle_h,ttl_length,item_per_session,frequency
userId,,,,,,,,,,,,,,,,,,,,,
1599112,M,2018-09-09 05:12:58,Windows,Firefox,34,0,0,1,1,1,...,0,0,1,0,2,2018-10-05 22:16:33,641.059722,8719.39682,17.000000,0.003120
1684541,M,2018-08-15 01:38:12,Macintosh,Chrome,429,0,11,11,0,0,...,0,3,21,0,3,2018-11-14 08:47:16,2191.151111,128715.47245,200.333333,0.001369
1433168,F,2018-04-16 01:07:26,Windows,Firefox,73,0,3,1,0,0,...,0,0,4,0,5,2018-11-15 10:24:34,5121.285556,18863.98000,11.400000,0.000976
1228521,M,2018-09-06 10:04:19,Windows,Chrome,37,0,0,1,1,1,...,0,0,2,0,2,2018-10-09 13:44:54,795.676389,8954.65473,16.500000,0.002514
1261287,F,2018-06-17 06:14:25,Windows,Chrome,493,0,10,19,0,0,...,1,6,29,1,5,2018-11-06 03:18:49,3405.073333,145789.09145,124.200000,0.001468


In [19]:
### df_users_base = df_users_base.drop(columns="user_lifecycle_h", errors="ignore")


In [20]:
"user_lifecycle_h" in df_users_base.columns

True

In [21]:
print(df_users_base["count_cancellation_confirmation"].unique())
df_users_base.rename(columns={"count_cancellation_confirmation":"churn_label"})

[1 0]


,gender,registration,operating_system,browser,num_unique_artists,count_about,count_add_friend,count_add_to_playlist,count_cancel,churn_label,...,count_submit_upgrade,count_thumbs_down,count_thumbs_up,count_upgrade,count_total_sessions,last_time,user_lifecycle_h,ttl_length,item_per_session,frequency
userId,,,,,,,,,,,,,,,,,,,,,
1749042,M,2018-08-08 13:22:21,Windows,Chrome,797,5,18,33,1,1,...,0,9,51,0,9,2018-10-21 01:16:24,1763.900833,256456.28661,51.444444,0.005102
1563081,F,2018-09-21 03:25:18,Macintosh,Chrome,437,1,5,14,0,0,...,0,5,25,0,6,2018-11-03 02:00:13,1030.581944,123645.65861,29.666667,0.005822
1697168,F,2018-09-08 13:48:25,Macintosh,Firefox,1117,1,22,44,0,0,...,1,16,83,7,34,2018-11-19 23:59:55,1738.191667,386582.82117,12.941176,0.019561
1222580,M,2018-08-16 02:31:00,Macintosh,Safari,1252,8,35,50,1,1,...,2,17,85,5,20,2018-10-30 23:17:30,1820.775000,452234.08024,25.650000,0.010984
1714398,F,2018-09-19 18:23:35,Windows,Chrome,1102,0,22,51,0,0,...,0,19,79,0,14,2018-11-19 05:16:55,1450.888889,384934.72168,24.857143,0.009649
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1494594,M,2018-08-18 18:07:54,iPhone,Safari,12,0,0,0,0,0,...,0,0,0,0,1,2018-11-19 19:52:00,2233.735000,3047.43992,18.000000,0.000448
1036641,F,2018-07-09 12:04:12,Windows,Chrome,71,0,3,1,0,0,...,0,0,12,3,1,2018-11-19 23:58:21,3203.902500,17586.59262,102.000000,0.000312
1110980,F,2018-08-31 14:17:48,Windows,Chrome,51,0,0,4,0,0,...,0,0,5,0,1,2018-11-19 23:55:01,1929.620278,14253.34974,70.000000,0.000518


In [22]:
df_users_base["avg_songs_session"] = df_users_base["count_nextsong"] / df_users_base["count_total_sessions"]

In [23]:
mask = df_users_base["user_lifecycle_h"] < 24
df_users_base[mask]

,gender,registration,operating_system,browser,num_unique_artists,count_about,count_add_friend,count_add_to_playlist,count_cancel,count_cancellation_confirmation,...,count_thumbs_down,count_thumbs_up,count_upgrade,count_total_sessions,last_time,user_lifecycle_h,ttl_length,item_per_session,frequency,avg_songs_session
userId,,,,,,,,,,,,,,,,,,,,,
1714714,F,2018-09-30 22:59:16,Macintosh,Safari,65,1,0,0,1,1,...,0,2,0,1,2018-10-01 09:25:13,10.432500,15332.72528,79.0,0.095854,64.0
1444644,F,2018-10-06 04:01:43,Macintosh,Safari,40,0,0,2,0,0,...,0,0,0,1,2018-10-07 02:00:54,21.986389,9873.95455,46.0,0.045483,39.0
1039566,M,2018-10-07 10:56:31,iPhone,Safari,73,0,0,1,0,0,...,2,5,0,1,2018-10-07 16:32:59,5.607778,20749.91474,104.0,0.178324,75.0
1594905,M,2018-10-09 02:53:40,Windows,Chrome,304,2,2,11,0,0,...,4,15,0,1,2018-10-10 02:30:24,23.612222,84829.05169,399.0,0.042351,338.0
1745396,F,2018-11-07 15:38:32,Macintosh,Firefox,24,0,0,1,0,0,...,0,1,0,1,2018-11-07 17:02:16,1.395556,5221.47530,33.0,0.716561,23.0
1822231,M,2018-11-11 16:22:57,Windows,Firefox,11,0,1,1,0,0,...,0,1,0,1,2018-11-11 16:57:05,0.568889,2193.78487,28.0,1.757812,10.0
1792581,F,2018-11-12 12:06:34,Macintosh,Safari,17,0,0,1,0,0,...,0,0,0,1,2018-11-12 13:11:23,1.080278,4095.83557,38.0,0.925688,17.0
1094942,M,2018-11-13 18:48:19,Macintosh,Safari,140,1,0,4,0,0,...,0,7,0,1,2018-11-14 05:19:34,10.520833,38257.17415,193.0,0.095050,147.0
1351427,F,2018-11-14 10:14:32,Windows,Chrome,30,0,0,0,0,0,...,0,0,1,1,2018-11-14 12:37:59,2.390833,8783.92155,50.0,0.418264,29.0


In [24]:
df_users_base.loc[[1714714]]

,gender,registration,operating_system,browser,num_unique_artists,count_about,count_add_friend,count_add_to_playlist,count_cancel,count_cancellation_confirmation,...,count_thumbs_down,count_thumbs_up,count_upgrade,count_total_sessions,last_time,user_lifecycle_h,ttl_length,item_per_session,frequency,avg_songs_session
userId,,,,,,,,,,,,,,,,,,,,,
1714714,F,2018-09-30 22:59:16,Macintosh,Safari,65,1,0,0,1,1,...,0,2,0,1,2018-10-01 09:25:13,10.4325,15332.72528,79.0,0.095854,64.0


In [ ]:
df_users_base["thumbs_ratio"] = df_users_base["count_thumbs_up"] / (df_users_base["count_thumbs_down"] + df_users_base["count_thumbs_up"])
df_users_base["thumbs_ratio"] = df_users_base["thumbs_ratio"].replace(np.inf, 0)
df_users_base["errors_per_session"] = df_users_base["count_error"] / df_users_base["count_total_sessions"]
df_users_base["ads_per_session"] = df_users_base["count_roll_advert"] / df_users_base["count_total_sessions"]

In [26]:
mask = df_users_base["thumbs_ratio"] ==0
df_users_base[mask]

,gender,registration,operating_system,browser,num_unique_artists,count_about,count_add_friend,count_add_to_playlist,count_cancel,count_cancellation_confirmation,...,count_upgrade,count_total_sessions,last_time,user_lifecycle_h,ttl_length,item_per_session,frequency,avg_songs_session,thumbs_ratio,ads_per_session
userId,,,,,,,,,,,,,,,,,,,,,
1441204,F,2018-09-05 01:06:30,Macintosh,Safari,29,0,0,1,0,0,...,1,2,2018-10-12 07:52:34,894.767778,7033.01263,18.0,0.002235,14.0,0.0,2.0
1072890,M,2018-09-18 06:45:18,Macintosh,Chrome,40,0,3,2,0,0,...,1,2,2018-10-01 18:28:56,323.727222,10032.43945,21.5,0.006178,19.5,0.0,1.5
1423609,F,2018-09-19 21:10:38,Macintosh,Chrome,81,1,2,2,0,0,...,1,5,2018-10-23 02:01:35,796.849167,19678.42068,8.6,0.006275,16.8,0.0,3.2
1322687,F,2018-08-24 15:45:20,Macintosh,Safari,6,0,0,0,1,1,...,0,2,2018-10-19 16:22:37,1344.621389,1151.89325,3.5,0.001487,2.5,0.0,0.5
1093443,F,2018-09-25 07:58:48,Windows,Firefox,20,0,0,0,1,1,...,0,2,2018-10-03 05:15:29,189.278056,5052.07302,11.0,0.010566,9.5,0.0,0.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1348111,F,2018-09-25 13:54:09,Windows,Chrome,13,0,3,0,0,0,...,0,1,2018-11-09 14:35:14,1080.684722,3539.09006,20.0,0.000925,12.0,0.0,0.0
1968013,F,2018-01-28 01:01:12,Macintosh,Chrome,23,0,0,2,0,0,...,0,1,2018-11-11 05:03:33,6892.039167,5872.94305,27.0,0.000145,23.0,0.0,0.0
1304158,F,2018-11-19 01:07:16,Macintosh,Chrome,28,0,2,1,0,0,...,0,1,2018-11-19 12:08:40,11.023333,6712.33386,59.0,0.090717,27.0,0.0,3.0
